In [42]:
class table_products():
    #create a table with the products information
    def __init__(self) -> None:
        self.prepare()
        self.organize_products()
        self.create_clusters()
        self.create_primary_key()

    def prepare(self):
        from A01_TOOL_BOX import tool_box
        self.ToolBox=tool_box()

    def organize_products(self):
        #import data
        self.tab_products=self.ToolBox.data
        #convert some columns into number type
        self.tab_products['Quantity Ordered']=self.tab_products['Quantity Ordered'].map(lambda x:int(x))
        self.tab_products['Price Each']=self.tab_products['Price Each'].map(lambda x:float(x))
        #group the diferent products
        self.tab_products=self.tab_products[['Product','Quantity Ordered','Price Each']].groupby(['Product']).sum()
        #get the average price. States it as the target price for marging calcules.
        self.tab_products['Target_price']=self.tab_products['Price Each']/self.tab_products['Quantity Ordered']
        self.tab_products['Target_price']=self.tab_products['Target_price'].map(lambda x:round(x,2))
        #state a minimum quantity to buy, units per pack. Based on the Quantity ordered
        self.tab_products['units/pack']=self.tab_products['Quantity Ordered'].map(lambda x:int(x/30))
        self.tab_products.drop(inplace=True,columns=['Quantity Ordered','Price Each'])


    def create_clusters(self):
        #Classify the products in clusters
        reference={
            'Monitor':['Monitor','TV'],
            'Others':['Batteries','Wired','Cable','Headphones'],
            'Phone':['Phone'],
            'home_appliances':['Dryer','Washing'],
            'Laptop':['Laptop']
        }
        self.tab_products['cluster']=self.tab_products.index
        for i in reference:
            for j in reference[i]:
                self.tab_products['cluster']=self.tab_products['cluster'].map(lambda x:i if x.find(j)!=-1 else x)
                

    def create_primary_key(self):
        #create an ID to each product to use as primary key
        reference={
            'Monitor':'01',
            'Others':'02',
            'Phone':'03',
            'home_appliances':'04',
            'Laptop':'05'
        }
        self.tab_products.reset_index(inplace=True)
        self.tab_products['ID_products']=self.tab_products['cluster']
        
        for i in reference:
            self.tab_products['ID_products']=self.tab_products['ID_products'].map(lambda x:reference[i] if x==i else x)
        self.tab_products['ID_products']='9'+self.tab_products['ID_products']+self.tab_products.index.map(lambda x:str(x).zfill(3))
        self.tab_products=self.tab_products


        display(self.tab_products)
table_products()

,Product,Target_price,units/pack,cluster,ID_products
0,20in Monitor,108.60,13,Monitor,901000
1,27in 4K Gaming Monitor,388.61,18,Monitor,901001
2,27in FHD Monitor,149.38,24,Monitor,901002
3,34in Ultrawide Monitor,378.24,21,Monitor,901003
4,AA Batteries (4-pack),2.81,94,Others,902004
5,AAA Batteries (4-pack),2.03,97,Others,902005
6,Apple Airpods Headphones,149.02,50,Others,902006
7,Bose SoundSport Headphones,99.37,42,Others,902007
8,Flatscreen TV,299.35,15,Monitor,901008
9,Google Phone,600.00,19,Phone,903009
